In [ ]:
import time

import numpy as np
import pandas as pd

from scipy.stats import mode

from tqdm import tqdm

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

BASE_URI = "..\\..\\DATA\\kredit\\"  # "D:\\Dropbox\\Aktuális munkák\\OH\\Kreditelorehaladas\\ADAT\\"

# Preparing

In [ ]:
def getcol(feature, yearcode):
    return ('HKI_{feature}_{yearcode}'
            .format(feature=feature, yearcode=yearcode))


def yearcode_from_date(date):
    if pd.isnull(date):
        return '9999'
    
    month = int(date.month)
    year = int(date.year) - int(month < 8)
    sem = '2' if month < 8 else '1'

    return '{:0>2}2{}'.format(str(year)[-2:], sem)


def reg_yearcode_from_date(date, start, end):
    if pd.isnull(date):
        return '9999'
    
    month = int(date.month)
    year = int(date.year) - int(start <= month <= end)
    sem = '2' if start <= month <= end else '1'
    
    return '{:0>2}2{}'.format(str(year)[-2:], sem)


def get_vegzes_kat(row, yearcode):
    start = int(reg_yearcode_from_date(row.HKPZ_KEZDDAT, 2, 7))
    end = int(reg_yearcode_from_date(row.HKPZ_VEGDAT, 2, 7))
    absz = int(reg_yearcode_from_date(row.HKPZ_ABSZ_DAT, 2, 7))
    vizsg = int(reg_yearcode_from_date(row.HKPZ_KIVIZSGA_DAT, 2, 7))
    adminaktiv = not pd.isnull(row[getcol('KPZI_ID', yearcode)])
    act = int(yearcode)
    
    if act < start and not adminaktiv:
        return 'nemkezdte'
    
    if act > vizsg and not adminaktiv:
        return 'zarovizsga'
    
    if act > absz and not adminaktiv:
        return 'abszolutorium'
        
    if act > end and not adminaktiv:
        return 'egyeb'
    
    statusz = row[getcol('STATUSZ', yearcode)]
    statusz = statusz.strip() if pd.notnull(statusz) else 'NaN'
    if statusz == 'A' and adminaktiv:
        return 'aktiv'
    if statusz == 'P' and adminaktiv:
        return 'passziv'
    
    return 'admin_hiba'


def get_credit_cat(credit):
    if pd.isnull(credit):
        return 'NaN'
    if credit < 0:
        return '<0'
    if credit == 0:
        return '0'
    if credit < 36:
        return '1-35'
    if credit < 61:
        return '36-60'
    
    return '>60'


def count_decreases(row, tipus, yearcodes):
    decs = 0
    for act, prev in zip(yearcodes[1:], yearcodes[:-1]):
        actcol = row[getcol(f'OSSZ_{tipus}_KREDIT', act)]
        prevcol = row[getcol(f'OSSZ_{tipus}_KREDIT', prev)]
        if pd.isnull(actcol):
            break
        if actcol < prevcol:
            decs += 1
    return decs


def sum_credits(row, minyear, col):
    yearcode = reg_yearcode_from_date(row[col], 2, 7)
    if int(yearcode) < int(minyear) or int(yearcode) > 1622:
        return 'null'
        
    credits = (row[getcol('OSSZ_MEGSZ_KREDIT', yearcode)] + 
               row[getcol('OSSZ_ELISM_KREDIT', yearcode)])
    target = (row.KPZE_KPZIDO or row.HKPZ_OSSZIDO) * 30
    ratio = credits / target
    
    if 1. <= ratio <= 1.1:
        return 'normal'
    
    if ratio > 1.1:
        return 'magas'
    
    return 'alacsony'


def aggregate(df, yearcode, grouping):
    out = {}
    for tipus in ['MEGSZ', 'ELISM']:
        out[f'{tipus}_STAT_{yearcode}'] = [
            df
            .groupby(grouping + [f'STATUSZ_{yearcode}'])
            [f'{tipus}_KREDIT_{yearcode}']
            .agg(['count', 'min', 'mean', 'median', 'max'])
        ]
        
        out[f'{tipus}_KATSTAT_{yearcode}'] = [
            df
            .groupby(grouping + [f'STATUSZ_{yearcode}', f'{tipus}_KREDIT_KAT_{yearcode}'])
            .HLG_SK.count()
            .to_frame()
            .rename(columns={'HLG_SK': 'occurences'})
        ]
        
    return out


def sum_decrease(group, semesters):
    df = pd.DataFrame({
        'type': ['megszerzett', 'elismert'],
        'occurences': [group[f'MEGSZ_DECREASE'].sum(),
                       group[f'ELISM_DECREASE'].sum()],
        'N': [group[f'MEGSZ_DECREASE'].count() * semesters,
              group[f'ELISM_DECREASE'].count() * semesters],
    }).set_index('type')
    df['ratio'] = (df.occurences / df.N).apply(lambda x: '{:.2f}%'.format(x*100))
    return df[['occurences', 'N', 'ratio']]


def count_credit_ratio(group, tipus):
    df = group.groupby(f'KREDIT_{tipus}').HLG_SK.count().to_frame().rename(columns={'HLG_SK': 'occurences'})
    df['N'] = df.occurences.sum()
    df['ratio'] = (df.occurences / df.N).apply(lambda x: '{:.2f}%'.format(x*100))
    return df[['occurences', 'N', 'ratio']]


def compute_extra_columns(BASE, yearcodes):
    for yearcode in yearcodes:
        BASE[f'STATUSZ_{yearcode}'] = BASE.apply(get_vegzes_kat, args=(yearcode,), axis=1)
        
    BASE[f'MEGSZ_KREDIT_{yearcodes[0]}'] = BASE[getcol('OSSZ_MEGSZ_KREDIT', yearcodes[0])]
    BASE[f'MEGSZ_KREDIT_KAT_{yearcodes[0]}'] = BASE[f'MEGSZ_KREDIT_{yearcodes[0]}'].apply(get_credit_cat)

    BASE[f'ELISM_KREDIT_{yearcodes[0]}'] = BASE[getcol('OSSZ_ELISM_KREDIT', yearcodes[0])]
    BASE[f'ELISM_KREDIT_KAT_{yearcodes[0]}'] = BASE[f'ELISM_KREDIT_{yearcodes[0]}'].apply(get_credit_cat)

    for act, prev in zip(yearcodes[1:], yearcodes[:-1]):
        BASE[f'MEGSZ_KREDIT_{act}'] = (BASE[getcol('OSSZ_MEGSZ_KREDIT', act)]
                                       - BASE[getcol('OSSZ_MEGSZ_KREDIT', prev)].fillna(0))
        BASE[f'MEGSZ_KREDIT_KAT_{act}'] = BASE[f'MEGSZ_KREDIT_{act}'].apply(get_credit_cat)

        BASE[f'ELISM_KREDIT_{act}'] = (BASE[getcol('OSSZ_ELISM_KREDIT', act)]
                                       - BASE[getcol('OSSZ_ELISM_KREDIT', prev)].fillna(0))
        BASE[f'ELISM_KREDIT_KAT_{act}'] = BASE[f'ELISM_KREDIT_{act}'].apply(get_credit_cat)
        
    BASE['MEGSZ_DECREASE'] = BASE.apply(count_decreases, axis=1, tipus='MEGSZ', yearcodes=yearcodes)
    BASE['ELISM_DECREASE'] = BASE.apply(count_decreases, axis=1, tipus='ELISM', yearcodes=yearcodes)
    
    BASE['KREDIT_ABSZ'] = BASE.apply(sum_credits, axis=1, minyear=yearcodes[0], col='HKPZ_ABSZ_DAT')
    BASE['KREDIT_VIZSG'] = BASE.apply(sum_credits, axis=1, minyear=yearcodes[0], col='HKPZ_KIVIZSGA_DAT')
    
    return BASE


def export_credit_basestats(BASE, yearcodes, filename):
    with pd.ExcelWriter(filename) as writer:
        for yearcode in yearcodes:
            for tipus in ['MEGSZ', 'ELISM']:
                tmp = (BASE
                       .groupby(f'STATUSZ_{yearcode}')
                       [f'{tipus}_KREDIT_{yearcode}']
                       .agg(['count', 'min', 'mean', 'median', 'max']))
                tmp['N'] = BASE[f'STATUSZ_{yearcode}'].count()
                tmp['feltöltöttség'] = (tmp['count'] / tmp.N * 100).apply(lambda x: '{:.2f}%'.format(x))
                (tmp
                 [['N', 'count', 'feltöltöttség','min', 'mean', 'median', 'max']]
                 .to_excel(writer, sheet_name=f'{tipus}_KREDITSTAT_{yearcode}'))

                (BASE
                 .groupby([f'STATUSZ_{yearcode}', f'{tipus}_KREDIT_KAT_{yearcode}'])
                 .HLG_SK.count()
                 .to_frame()
                 .rename(columns={'HLG_SK': 'elofordulas'})
                 .to_excel(writer, sheet_name=f'{tipus}_KREDIT_KATSTAT_{yearcode}'))
                
                
def export_credit_detailstats(BASE, yearcodes, filename):
    with pd.ExcelWriter(filename) as writer:
        for yearcode in yearcodes:
            dfs = aggregate(BASE, yearcode, [])
            for grouping in ['SZER_KOD', 'HKPZ_MUNKAREND_SZO_HNEV', 
                             'HKPZ_KEL_TIPUS_NEV', 'HKI_FINANSZIROZAS_SZO_HNEV_{}']:
                grouping = grouping.format(yearcode)
                result = aggregate(BASE, yearcode, [grouping])
                for key, value in result.items():
                    dfs[key] += value

            for key, values in dfs.items():
                row = 0
                for df in values:
                    df.to_excel(writer, sheet_name=key, startrow=row , startcol=0)   
                    row = row + len(df.index) + 4
                    
                    
def export_credit_decrease(BASE, yearcodes, filename):
    row = 0
    with pd.ExcelWriter(filename) as writer:
        df = BASE.groupby(lambda _ : True).apply(sum_decrease, semesters=len(yearcodes)-1)
        df.to_excel(writer, sheet_name='csokkenes', startrow=row , startcol=0)
        row = row + len(df.index) + 4
        for grouping in ['SZER_KOD', 'HKPZ_MUNKAREND_SZO_HNEV', 'HKPZ_KEL_TIPUS_NEV']:
            df = BASE.groupby(grouping).apply(sum_decrease, semesters=len(yearcodes)-1)
            df.to_excel(writer, sheet_name='csokkenes', startrow=row , startcol=0)
            row = row + len(df.index) + 4
            
            
def export_credit_counts(BASE, filename):
    with pd.ExcelWriter(filename) as writer:
        for tipus in ['ABSZ', 'VIZSG']:
            row = 0
            df = BASE.groupby(lambda _ : True).apply(count_credit_ratio, tipus=tipus)
            df.to_excel(writer, sheet_name=tipus, startrow=row , startcol=0)
            row = row + len(df.index) + 4

            for grouping in ['SZER_KOD', 'HKPZ_MUNKAREND_SZO_HNEV', 'HKPZ_KEL_TIPUS_NEV']:
                df = BASE.groupby(grouping).apply(count_credit_ratio, tipus='ABSZ')
                df.to_excel(writer, sheet_name=tipus, startrow=row , startcol=0)
                row = row + len(df.index) + 4
                
                
def export_credit_progres(BASE, yearcodes, filename):
    with pd.ExcelWriter(filename) as writer:
        for tipus in ['MEGSZ', 'ELISM']:
            for yc in yearcodes:
                (BASE
                 .groupby(getcol('AKTIV_EVFOLYAM', yc))
                 [getcol(f'OSSZ_{tipus}_KREDIT', yc)]
                 .describe()
                 .to_excel(writer, sheet_name=f'{tipus}_{yc}'))

                
def analyse_one(BASE_URI, level, startyear, required_metrics='all'):
    all_metrics = ['base', 'details', 'decrease', 'counts', 'progress']
    if required_metrics == 'all':
        metrics = all_metrics
    else:
        metrics = [metric for metric in required_metrics 
                   if metric in all_metrics]
    
    print('Selected metrics: {}'.format(', '.join(metrics)))
    start_time = time.time()
    filename = f'861_1107_{startyear}_{level}_v1.xlsx'
    print(f'Reading {filename}...', end=' ')
    process_start = time.time()
    BASE = pd.read_excel(BASE_URI + filename)
    print('Done in {:.2f} sec.'.format(time.time() - process_start))
    if len(BASE) == 0:
        return
    
    print('Computing features...', end=' ')
    process_start = time.time()
    base_year = str(startyear)[2:]
    yearcodes = [f'{y:0>2}2{s}' for y in range(int(base_year), 17) for s in [1, 2]]
    BASE = compute_extra_columns(BASE, yearcodes)
    print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    if 'base' in metrics:
        filename = f'kredit_statisztikak_{startyear}_{level}.xlsx'
        print(f'Generating {filename}...', end=' ')
        process_start = time.time()
        export_credit_basestats(BASE, yearcodes, BASE_URI + filename)
        print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    if 'details' in metrics:
        filename = f'kredit_statisztikak_megbontva_{startyear}_{level}.xlsx'
        print(f'Generating {filename}...', end=' ')
        process_start = time.time()
        export_credit_detailstats(BASE, yearcodes, BASE_URI + filename)
        print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    if 'decrease' in metrics:
        filename = f'kredit_csokkenes_{startyear}_{level}.xlsx'
        print(f'Generating {filename}...', end=' ')
        process_start = time.time()
        export_credit_decrease(BASE, yearcodes, BASE_URI + filename)
        print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    if 'counts' in metrics:
        filename = f'kreditszam_{startyear}_{level}.xlsx'
        print(f'Generating {filename}...', end=' ')
        process_start = time.time()
        export_credit_counts(BASE, BASE_URI + filename)
        print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    if 'progress' in metrics:
        filename = f'kredit_evfolyam_{startyear}_{level}.xlsx'
        print(f'Generating {filename}...', end=' ')
        process_start = time.time()
        export_credit_progres(BASE, yearcodes, BASE_URI + filename)
        print('Done in {:.2f} sec.'.format(time.time() - process_start))
    
    print('Process done, total time: {:.2f} sec'.format(time.time() - start_time))

---

# Run the process

In [ ]:
%%time
for year in [2009, 2011, 2016]:
    for level in 'BMOS':
        analyse_one(BASE_URI, level, year, ['details'])
        print('-'*80)